## Two Moon Classification with stochastic depth

* We will take the two moon dataset as a dummy dataset that's easy to visualize, and just qualitatively see how the uncertainty boundaries differ compared to dropout.
    * IF the results look good, we'll also try to implement other methods such as Laplace Approximation for visualization

In [ ]:
from functools import partial
import numpy as np
import torch
from torch import nn

import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from torchvision.ops import StochasticDepth
from tqdm import tqdm

from torch import Tensor
from typing import List, Tuple


### Two Moon dataset

In [ ]:
def split_dataset(X, Y, train_ratio=0.8):
    n = len(X)
    n_train = int(n * train_ratio)
    indices = torch.randperm(n)
    train_indices, test_indices = indices[:n_train], indices[n_train:]
    X_train, Y_train = X[train_indices], Y[train_indices]
    X_test, Y_test = X[test_indices], Y[test_indices]

    return X_train, Y_train, X_test, Y_test


def plot_two_moons(X, Y):
    plt.figure(figsize=(5, 5))
    for i in range(len(X)):
        plt.scatter(X[i, 0], X[i, 1], c="orange" if Y[i] == 1 else "green")
    # plt.axis("off")


X, Y = make_moons(n_samples=200, shuffle=True, noise=0.25, random_state=1)
X, Y = torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)

# plot_two_moons(X, Y)

### Bayesian Network Wrapper Class

In [ ]:
class BayesianNet(nn.Module):
    """
    Bayesian network class has two internal states, dropout_state and stochastic_depth_state.
    They overwrite the default behaviour of forward pass to enable switching between bayesian and deterministic modes.
    Toggling the states is done by calling set_bayes_mode method.
    """

    def __init__(self) -> None:
        super().__init__()
        self.dropout_state: bool = False
        self.stochastic_depth_state: bool = False

    def _toggle_dropout(self) -> None:
        # override the default behaviour of dropout layers
        if self.dropout_state:
            for m in self.modules():
                if m.__class__.__name__.startswith("Dropout"):
                    m.train()

    def _toggle_stochastic_depth(self) -> None:
        # override the default behaviour of stochastic depth layers
        if self.stochastic_depth_state:
            for m in self.modules():
                if m.__class__.__name__.startswith("StochasticDepth"):
                    m.train()

    def _set_dropout(self, state: bool) -> None:
        self.dropout_state = state

    def _set_stochastic_depth(self, state: bool) -> None:
        self.stochastic_depth_state = state

    def set_bayes_mode(self, state: bool, mode: str) -> None:
        if mode == "dropout":
            self._set_dropout(state)
        elif mode == "sd":
            self._set_stochastic_depth(state)
        else:
            print(f"Mode {mode} is not supported.")

    def forward(self, x: Tensor) -> Tensor:
        self._toggle_stochastic_depth()
        self._toggle_dropout()

        return self._forward_impl(x)

    def _forward_impl(self, x: Tensor) -> Tensor:
        raise NotImplementedError

### A simple residual network

In [ ]:
class _ResidualSdBlock(nn.Module):
    """Implements a simple x + lin(x) block"""

    def __init__(self, features: int, p: float = 0.5) -> None:
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(features, features), nn.BatchNorm1d(features), nn.ReLU()
        )
        self.p = p
        self.sd = StochasticDepth(p, mode="row")

    def forward(self, x: Tensor) -> Tensor:
        out = self.block(x)
        out = self.sd(out)
        # apply skip connection
        out = x + out
        return out
    
class SdNetwork(BayesianNet):
    def __init__(self, inp: int, oup: int, N: int, d_feat: int, p=0.5):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Linear(inp, d_feat),
            nn.BatchNorm1d(d_feat),
            nn.ReLU(),
            *[_ResidualSdBlock(d_feat, p=p) for _ in range(N)],
            nn.Linear(d_feat, oup),
            nn.Softmax(dim=-1)
        )

    def _forward_impl(self, x: Tensor) -> Tensor:
        return self.blocks(x)

### A simple dropout network

In [ ]:
class _ResidualDropoutBlock(nn.Module):
    """Implements a simple dropout(lin(x)) block"""

    def __init__(self, features: int, p: float = 0.5) -> None:
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(features, features),
            nn.BatchNorm1d(features),
            nn.Dropout1d(p=p),
            nn.ReLU(),
        )

    def forward(self, x: Tensor) -> Tensor:
        out = self.block(x)
        # apply skip connection
        out = x + out
        return out

class DONetwork(BayesianNet):
    def __init__(self, inp: int, oup: int, N: int, d_feat: int, p=0.5):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Linear(inp, d_feat),
            nn.BatchNorm1d(d_feat),
            nn.ReLU(),
            *[_ResidualDropoutBlock(d_feat, p=p) for _ in range(N)],
            nn.Linear(d_feat, oup),
            nn.Softmax(dim=-1)
        )

    def _forward_impl(self, x: Tensor) -> Tensor:
        return self.blocks(x)

### Bayesian forward pass

In [ ]:
def bayes_forward(x: Tensor, net: BayesianNet, mode: str, T: int) -> Tensor:
    net.eval()
    net.set_bayes_mode(True, mode)
    with torch.no_grad():
        y_logits_samples = net(x.repeat(T, 1))
        y_logits = y_logits_samples.mean(dim=0)

    return y_logits


def batch_bayes_forward(x_batch: Tensor, net: BayesianNet, mode: str, T: int) -> Tensor:
    """
    Performs bayesian forward pass in eval mode for a batch of input
    Args
    - net
    - x_batch: tensor with shape (n, m) containing n samples of m dimensional inputs
    - mode: 'dropout' or 'sd'
    - T: number of stochastic samples

    Returns
    - y_logits_batch: tensor with shape (n, c)
    """
    net.eval()
    net.set_bayes_mode(True, mode)
    n = x_batch.shape[0]
    with torch.no_grad():
        y_logits_samples = net(x_batch.repeat(T, 1))  # (Txn, m)
        y_logits_batch = y_logits_samples.reshape((T, n, -1))
    net.set_bayes_mode(False, mode)

    return y_logits_batch.mean(dim=0)


def batch_ensemble_forward(x_batch: Tensor, nets: List[nn.Module]) -> Tensor:
    for net in nets:
        net.eval()
    T = len(nets)
    # use the first network to determine the shape of the output
    y_logits = net(x_batch)  # n x c
    y_logits_samples = torch.empty(T, *y_logits.shape)  # T x n x c
    y_logits_samples[0, :] = y_logits
    # iterate over the rest of the network
    for i, net in enumerate(nets[1:]):
        y_logits_samples[i + 1, :] = net(x_batch)

    return y_logits_samples.mean(dim=0)  # n x c

### Train DO & sd networks

In [ ]:
# net:BayesianNet = DONetwork(inp=2, oup=2, N=2, d_feat=30, p=0.5)
net:BayesianNet = DONetwork(inp=2, oup=2, N=10, d_feat=10, p=0.5)
# net:BayesianNet = SdNetwork(inp=2, oup=2, N=10, d_feat=10, p=0.5)
net.train()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4)
criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()

history = []

for epoch in tqdm(range(500)):
    optimizer.zero_grad()
    y_logits = net(X)
    loss = criterion(y_logits.squeeze(1), Y.to(torch.long))
    loss.backward()
    optimizer.step()

    history.append(loss.item())

# plt.plot(history)
    
net.eval()
with torch.no_grad():
    net.set_bayes_mode(False, "sd")
    y_pred = net(X).argmax(dim=1)

# plot_two_moons(X, y_pred)

print(f"Accuracy: {(y_pred.squeeze() == Y).sum()/len(Y):.3f}")

### Qualitative view of confidence scores

In [ ]:
def compute_confidence(inp, forward_func, batch_size) -> Tensor:
    """Computes confidence score over the entire inp, handles batching"""
    py = torch.empty((len(inp), 1), dtype=torch.float)
    b = batch_size
    for i in tqdm(range(0, inp.shape[0], b)):
        x = inp[i : i + b]
        logits = forward_func(x)
        p, _ = torch.max(logits, dim=-1)
        py[i : i + b] = p.unsqueeze(-1)
    return py


def plot_confidence(net: BayesianNet, forward_func, grid_size=100, batch_size=10):
    net.eval()
    XX, YY = np.meshgrid(np.linspace(-3, 3, grid_size), np.linspace(-3, 3, grid_size))
    with torch.no_grad():
        inp = torch.FloatTensor(np.stack((XX.ravel(), YY.ravel())).T)
        # py, _ = torch.max(net(inp), dim=-1)
        py = compute_confidence(inp, forward_func, batch_size)
        conf = torch.where(py > 0.5, py, 1 - py)  ## THIS LINE COMPUTES THE CONFIDENCE
        ZZ = conf.reshape(XX.shape)

        plt.contourf(XX, YY, ZZ, cmap="PiYG", vmin=0.5, vmax=1.2)
        plt.colorbar(ticks=np.linspace(0.5, 1, 6))
        plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="autumn", edgecolor="k")
        plt.title("Confidence plot")


forward_func = partial(batch_bayes_forward, net=net, mode="dropout", T=30)
plot_confidence(net, forward_func, grid_size=100, batch_size=50)